In [6]:
import pandas as pd 
import numpy as np
from astroquery.svo_fps import SvoFps
import numpy as np

# SVO Filter Profile Service
tbl_swift = SvoFps.get_filter_list(facility='Swift', instrument='UVOT')  # UV MAGS
tbl_mcps = SvoFps.get_filter_list(facility='LCO', instrument='')  # MCPS MAGS 

def vega2abcorr(Z_vega,Z_AB=3631):
    # Vega is 3631 Jy
    return 2.5*np.log10(Z_AB/Z_vega)

Z_ABs = []
for uvfilter in ['UVW2','UVM2','UVW1']:
    row = tbl_swift[tbl_swift['filterID']==f'Swift/UVOT.{uvfilter}']
    Z_AB = vega2abcorr(row['ZeroPoint'].data[0])
    Z_ABs.append(Z_AB)

for ofilter in ['U','B','V','I']:
    row = tbl_mcps[tbl_mcps['filterID']==f'Misc/MCPS.{ofilter}']
    Z_AB = vega2abcorr(row['ZeroPoint'].data[0])
    Z_ABs.append(Z_AB)


smc_Av = 0.22
smc_extinction_coeff = [3.522767303652845, 3.062652780996107, 2.7436011426496876, 1.746851127566682, 1.4073996800012445, 1.0353852912271932, 0.5911705440475679]
smc_Alam = np.array(smc_extinction_coeff) * smc_Av


lmc_Av = 0.38
lmc_extinction_coeff = [2.644541680555541, 2.7233599880955124, 2.3536449902431045, 1.5634790597438197, 1.3170082045312625, 1.0333402844940784, 0.7366865234305091]
lmc_Alam = np.array(lmc_extinction_coeff) * lmc_Av



# Set up dataframe dict
df_dict = {'Filter (X)': ['UVW2', 'UVM2', 'UVW1', 'U', 'B', 'V', 'I'],
           '$A_{X}/A_{\nu}$$^1$ (LMC)': lmc_extinction_coeff,
           '$A_{X}/A_{\nu}$$^1$ (SMC)': smc_extinction_coeff,
           'Vega to AB': Z_ABs}
df = pd.DataFrame(df_dict)
df.to_csv('ExtinctionAndVega2AB.csv',index=False)
print(df.to_latex(index=False,float_format="%.3f"))

\begin{tabular}{lrrr}
\toprule
Filter (X) & $A_{X}/A_{
u}$$^1$ (LMC) & $A_{X}/A_{
u}$$^1$ (SMC) & Vega to AB \\
\midrule
UVW2 & 2.645 & 3.523 & 1.705 \\
UVM2 & 2.723 & 3.063 & 1.659 \\
UVW1 & 2.354 & 2.744 & 1.489 \\
U & 1.563 & 1.747 & 0.761 \\
B & 1.317 & 1.407 & -0.110 \\
V & 1.033 & 1.035 & -0.001 \\
I & 0.737 & 0.591 & 0.491 \\
\bottomrule
\end{tabular}

